In [ ]:
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

In [ ]:
# Pré-processamento dos Dados
# Separe as variáveis da seguinte forma:
# X: Todas as colunas, exceto classe.
# y: Coluna classe, que será a variável-alvo para a classificação.

# Separando as variáveis
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']

In [ ]:
# Configuração do Experimento
# Realize três experimentos, alterando o random_state para os valores 42, 17 e 24. Para cada experimento, aplique o KNN com k=3 e k=5.

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

random_states = [42, 17, 24]
k_values = [3, 5]

for random_state in random_states:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
  for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Random State: {random_state}, k: {k}, Acurácia: {accuracy}")

Random State: 42, k: 3, Acurácia: 0.7407407407407407
Random State: 42, k: 5, Acurácia: 0.7407407407407407
Random State: 17, k: 3, Acurácia: 0.6481481481481481
Random State: 17, k: 5, Acurácia: 0.6666666666666666
Random State: 24, k: 3, Acurácia: 0.7407407407407407
Random State: 24, k: 5, Acurácia: 0.6666666666666666


In [ ]:
# Estrutura de Cross-Validation com KFold
from sklearn.model_selection import KFold
import numpy as np

for random_state in random_states:
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    for k in k_values:
        accuracies = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)

        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        print(f"Random State: {random_state}, k: {k}, Acurácia Média: {mean_accuracy:.4f}, Desvio Padrão: {std_accuracy:.4f}")

Random State: 42, k: 3, Acurácia Média: 0.7036, Desvio Padrão: 0.1392
Random State: 42, k: 5, Acurácia Média: 0.6641, Desvio Padrão: 0.1001
Random State: 17, k: 3, Acurácia Média: 0.6987, Desvio Padrão: 0.1040
Random State: 17, k: 5, Acurácia Média: 0.6876, Desvio Padrão: 0.1121
Random State: 24, k: 3, Acurácia Média: 0.7127, Desvio Padrão: 0.0968
Random State: 24, k: 5, Acurácia Média: 0.7127, Desvio Padrão: 0.0940


In [ ]:
# Análise dos Resultados
# Calculando a média e o desvio padrão da acurácia para cada valor de k em cada experimento.
# Identificando o melhor valor de k para cada random_state.
# Caso haja empate nas acurácias médias para k=3 e k=5, registrando o empate nos resultados.

results = {}

for random_state in random_states:
    results[random_state] = {}
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    for k in k_values:
        accuracies = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)

        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        results[random_state][k] = (mean_accuracy, std_accuracy)
        print(f"Random State: {random_state}, k: {k}, Acurácia Média: {mean_accuracy:.4f}, Desvio Padrão: {std_accuracy:.4f}")

for random_state, k_results in results.items():
    best_k = None
    best_accuracy = -1
    for k, (mean_accuracy, _) in k_results.items():
        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_k = k
        elif mean_accuracy == best_accuracy:
            best_k = "Empate entre k=3 e k=5"
    print(f"Melhor k para Random State {random_state}: {best_k} com acurácia {best_accuracy:.4f}")

Random State: 42, k: 3, Acurácia Média: 0.7036, Desvio Padrão: 0.1392
Random State: 42, k: 5, Acurácia Média: 0.6641, Desvio Padrão: 0.1001
Random State: 17, k: 3, Acurácia Média: 0.6987, Desvio Padrão: 0.1040
Random State: 17, k: 5, Acurácia Média: 0.6876, Desvio Padrão: 0.1121
Random State: 24, k: 3, Acurácia Média: 0.7127, Desvio Padrão: 0.0968
Random State: 24, k: 5, Acurácia Média: 0.7127, Desvio Padrão: 0.0940
Melhor k para Random State 42: 3 com acurácia 0.7036
Melhor k para Random State 17: 3 com acurácia 0.6987
Melhor k para Random State 24: Empate entre k=3 e k=5 com acurácia 0.7127
